## 在线测评


利用 对给定的**网易云音乐精彩评论**进行：

    1、数据预处理（包括压缩去词、短句过滤、去停用词等操作）

    2、利用SnowNLP对预处理后的数据进行情感分析

    3、分析预处理后评论的情感倾向，并进行主题分析

## 在线测评示例代码

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = r'./网易云音乐精彩评论.csv'
df = pd.read_csv(path, encoding = 'gbk')
df

,用户Id,昵称,评论内容,点赞数
0,37408634,你是不沉的胖子,2004年是神奇的一年，那年有周杰伦的七里香，飞儿乐团的我们的爱，张韶涵的寓言，蔡依林的爱情...,203673
1,17746030,黄昏落雨时,随机到这首歌时真真切切得有种时光倒流的感觉。仿佛又看到10年前那个懵懂的自己；那个面对西瓜地...,79136
2,41306320,武安李牧,我觉得这是周杰伦最经典的情歌,65522
3,52840531,Hantzzz,记得小学听七里香时，每当听到“轻吻你倔强的嘴”的时候。就觉得的好露骨，好尴尬，生怕爸爸妈妈在...,63940
4,5621522,理想乌托邦mei,周杰伦应该是网易云音乐唯一一个每首歌的评论都过百的歌手,55923
...,...,...,...,...
170,99076801,小屋麦先生,在四川待了三年，是没有听过这首歌的三年。后来离开了四川，听到这首歌时，心里的某个位置，突然就...,34480
171,128393417,跑跑跑杨,因为李志爱上南京，因为赵雷爱上成都，路上的人，总把他乡当故乡，那就和解吧，那就歌唱吧，总之，...,31857
172,37440630,Hauntin,赵雷的歌朴实无华，都是对生活的体会和领悟。都是自己真实的经历，所以给人的感觉真实。能引起共鸣...,31780
173,56011501,特立独行的种猪,这首歌反复的唱都唱不厌，他在许多现场都唱过。我前些天在我的电台也唱过，虽然我唱的不咋地，也算...,23947


In [3]:
# 查看缺失值
df.isnull().sum()

用户Id    0
昵称      0
评论内容    0
点赞数     0
dtype: int64

In [4]:
# 机械压缩去词，去除重复累赘词语
# 定义函数
def str_unique(raw_str, reverse=False):
    if reverse:
        raw_str = raw_str[::-1]
    res_str = ''
    for i in raw_str:
        if i not in res_str:
            res_str += i
    if reverse:
        res_str = res_str[::-1]
    return res_str

In [5]:
# 使用 apply 方法应用函数；
ser1 = df.iloc[:, 2].apply(str_unique) # 这时，因为索引了第一列，所以结果成了 Series；
print('df2', type(ser1)) # <class 'pandas.core.series.Series'>
df2 = pd.DataFrame(ser1.apply(str_unique, reverse=True)) # 再次生成 DataFrame；
print('机械压缩去词后：')
print(len(df2))
print(type(df2))
print('------------------')

df2 <class 'pandas.core.series.Series'>
机械压缩去词后：
175
<class 'pandas.core.frame.DataFrame'>
------------------


In [6]:
# 短句过滤，过滤评论字数少于 4 个符的评论
df3 = df2[df2.iloc[:, 0].apply(len) >= 4]
print('短句过滤后：')
print(len(df3))
print('------------------')

短句过滤后：
175
------------------


In [7]:
# 情感分析
from snownlp import SnowNLP # 情感分析语言处理库
# 语义积极的概率，越接近 1 情感表现越积极
coms = df3.iloc[:, 0].apply(lambda x: SnowNLP(x).sentiments)
print('情感分析后：')
positive_df = df3[coms >= 0.9] # 特别喜欢的
negative_df = df3[coms < 0.1] # 不喜欢的
print('特别喜欢的')
print(positive_df)
print('------------------')
print('不喜欢的')
print(negative_df)

情感分析后：
特别喜欢的
                                                  评论内容
0    204年是神奇的一，那有周杰伦七里香飞儿乐团我们爱张韶涵寓言蔡依林情三十六计俊江南梁静茹宁夏...
1    随机到这首歌时真切得有种光倒流的感觉。仿佛又看10年前那个懵懂自己；面对西瓜地，背靠屠宰场乡...
3    记得小学听七里香时，每当到“轻吻你倔强的嘴”候。就觉好露骨尴尬生怕爸妈在旁边而现我已然变成了...
5                                  十年前。磁带随身听初恋青春阳光雨下整夜
6                                          这他妈才是老子的青春。
..                                                 ...
169    因为一首歌怀念座城。曾经的天府广场武侯祠锦里杜甫草堂青羊宫春熙路宽窄巷子昭觉寺文殊院都成了回忆
171     因为李志爱上南京，赵雷成都路的人总把他乡当故那就和解吧歌唱之你我要好活着愿走更多听一个有用。
172  赵雷的歌朴实无华，都是对生活体会和领悟。自己真经历所以给人感觉能引起共鸣第一次听成就深爱上了...
173  这首歌反复的唱都不厌，他在许多现场过。我前些天电台也虽然咋地算是对赵雷崇敬了有个朋友几刚去成...
174  一位江苏男孩因为成都女而喜欢上了这首歌，不异地恋但是。他们在起274天5原谅能去直陪你想：三...

[106 rows x 1 columns]
------------------
不喜欢的
                                                  评论内容
20   有没85前听杰伦的来这？我4，20年开始刚上高一现在都是俩孩爹了再谢老歌好像回到那时候晚自习...
77   现在的我，一个月工资30每天早上8.到晚。这消费流下西安 合租房子9贷1嗯剩公交电话生活不敢...
80   昨天没有买到开往成都的火车票 和老妈在旅馆了个标间一直后半夜睡着早上我问她晚是不失眠说第次这...
96                           有没人把专辑上的眼睛看成嘴。我大晚到心一凉差点吓尿
111 

In [8]:
## 分词分析
import jieba
my_cut = lambda s: ' '.join(jieba.cut(s)) # 自定义简单分词函数
positive_ser = positive_df.iloc[:, 0].apply(my_cut) # 通过“广播机制”分词，加快速度
negative_ser = negative_df.iloc[:, 0].apply(my_cut)
print('大于 0.5---正面数据---分词')
print(positive_ser)
print('小于 0.5---负面数据---分词')
print(negative_ser)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\TAOZHI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.652 seconds.
Prefix dict has been built successfully.


大于 0.5---正面数据---分词
0      204 年 是 神奇 的 一 ， 那有 周杰伦 七里香 飞儿 乐团 我们 爱 张韶涵 寓言 ...
1      随机 到 这 首歌 时 真切 得 有种 光 倒流 的 感觉 。 仿佛 又 看 10 年前 那...
3      记得 小学 听 七里香 时 ， 每当 到 “ 轻吻 你 倔强 的 嘴 ” 候 。 就 觉好 ...
5                           十年 前 。 磁带 随身听 初恋 青春 阳光 雨下 整夜
6                                    这 他 妈 才 是 老子 的 青春 。
                             ...                        
169    因为 一首歌 怀念 座城 。 曾经 的 天府广场 武侯祠 锦里 杜甫 草堂 青羊宫 春熙路 ...
171    因为 李志爱 上 南京 ， 赵雷 成都路 的 人 总 把 他 乡 当 故 那 就 和解 吧 ...
172    赵雷 的 歌 朴实无华 ， 都 是 对 生活 体会 和 领悟 。 自己 真 经历 所以 给 ...
173    这 首歌 反复 的 唱 都 不厌 ， 他 在 许多 现场 过 。 我 前些天 电台 也 虽然...
174    一位 江苏 男孩 因为 成都 女 而 喜欢 上 了 这 首歌 ， 不 异地 恋 但是 。 他...
Name: 评论内容, Length: 106, dtype: object
小于 0.5---负面数据---分词
20     有 没 85 前 听 杰伦 的 来 这 ？ 我 4 ， 20 年 开始 刚上 高一 现在 都...
77     现在 的 我 ， 一个月 工资 30 每天 早上 8 . 到 晚 。 这 消费 流下 西安 ...
80     昨天 没有 买 到 开往 成都 的 火车票   和 老妈 在 旅馆 了 个 标间 一直 后半...
96              有 没人 把 专辑 上 的 眼睛 看成 嘴 。 我大晚 到 心一凉 差点 吓 尿
111    我妈 中午 修 机器 的 时候 ， 手指头 被 压断 了 一节 到 医院 让 把 那 给 扔...
141    正在 看

In [9]:
stop_list = r'./stoplist.txt' # 我的停用词文件是在同级目录存放
stops = pd.read_csv(stop_list, encoding='gbk', header=None, sep='tipdm', engine='python')
# sep 设置分割词，由于 csv 默认以半角逗号为分割此，而该词恰好在停用词表中，因此会导致读取出错
# 所以解决办法是手动设置一个不存在的分割词，如 tipdm；
stops = [' ', ''] + list(stops[0]) # pandas 自动过滤了空格符，这里手动添加
positive_df = pd.DataFrame(positive_ser)
negative_df = pd.DataFrame(negative_ser)
positive_df['分割后'] = positive_df['评论内容'].apply(lambda s: s.split(' ')) # 定义一个分割函数，然后用 apply 广播
positive_df['去停用词后'] = positive_df['分割后'].apply(lambda x: [i for i in x if i.encode('utf-8') not in stops])
negative_df['分割后'] = negative_df['评论内容'].apply(lambda s: s.split(' ')) # 定义一个分割函数，然后用 apply 广播
negative_df['去停用词后'] = negative_df['分割后'].apply(lambda x: [i for i in x if i.encode('utf-8') not in stops])
print('去停用词后：positive_df')
print(positive_df)
print('------------------')
print('去停用词后：negative_df')
print(negative_df)

去停用词后：positive_df
                                                  评论内容  \
0    204 年 是 神奇 的 一 ， 那有 周杰伦 七里香 飞儿 乐团 我们 爱 张韶涵 寓言 ...   
1    随机 到 这 首歌 时 真切 得 有种 光 倒流 的 感觉 。 仿佛 又 看 10 年前 那...   
3    记得 小学 听 七里香 时 ， 每当 到 “ 轻吻 你 倔强 的 嘴 ” 候 。 就 觉好 ...   
5                         十年 前 。 磁带 随身听 初恋 青春 阳光 雨下 整夜   
6                                  这 他 妈 才 是 老子 的 青春 。   
..                                                 ...   
169  因为 一首歌 怀念 座城 。 曾经 的 天府广场 武侯祠 锦里 杜甫 草堂 青羊宫 春熙路 ...   
171  因为 李志爱 上 南京 ， 赵雷 成都路 的 人 总 把 他 乡 当 故 那 就 和解 吧 ...   
172  赵雷 的 歌 朴实无华 ， 都 是 对 生活 体会 和 领悟 。 自己 真 经历 所以 给 ...   
173  这 首歌 反复 的 唱 都 不厌 ， 他 在 许多 现场 过 。 我 前些天 电台 也 虽然...   
174  一位 江苏 男孩 因为 成都 女 而 喜欢 上 了 这 首歌 ， 不 异地 恋 但是 。 他...   

                                                   分割后  \
0    [204, 年, 是, 神奇, 的, 一, ，, 那有, 周杰伦, 七里香, 飞儿, 乐团,...   
1    [随机, 到, 这, 首歌, 时, 真切, 得, 有种, 光, 倒流, 的, 感觉, 。, ...   
3    [记得, 小学, 听, 七里香, 时, ，, 每当, 到, “, 轻吻, 你, 倔强, 的,...   
5              [十年, 前, 。, 磁带, 随身听, 初恋, 青春, 阳光, 雨下, 整夜

In [10]:
## 对 评论 进行情感的主题分析

from gensim import corpora, models
# 正面主题分析
pos_dict = corpora.Dictionary(positive_df['去停用词后'])
pos_corpus = [pos_dict.doc2bow(i) for i in positive_df['去停用词后']]
pos_lda = models.LdaModel(pos_corpus, num_topics=3, id2word=pos_dict)
print('#正面主题分析')
for i in range(3):
    print('topic', i)
    print(pos_lda.print_topic(i)) # 输出每个主题
# 负面主题分析
neg_dict = corpora.Dictionary(negative_df['去停用词后']) # 建立词典
neg_corpus = [neg_dict.doc2bow(i) for i in negative_df['去停用词后']] # 建立语料库
neg_lda = models.LdaModel(neg_corpus, num_topics=3, id2word=neg_dict) # LDA 模型训练
print('#负面主题分析')
for i in range(3):
    print('topic', i)
    print(neg_lda.print_topic(i)) # 输出每个主题

#正面主题分析
topic 0
0.017*"的" + 0.017*"" + 0.011*"，" + 0.011*"。" + 0.010*"了" + 0.009*"是" + 0.009*"我" + 0.008*"你" + 0.008*"在" + 0.006*"]"
topic 1
0.021*"，" + 0.019*"的" + 0.017*"。" + 0.014*"了" + 0.011*"我" + 0.008*"你" + 0.007*"在" + 0.007*"是" + 0.007*"这" + 0.007*"就"
topic 2
0.012*"了" + 0.011*"，" + 0.011*"的" + 0.009*"" + 0.007*"。" + 0.007*"我" + 0.006*"在" + 0.005*"都" + 0.005*"你" + 0.005*"听"
#负面主题分析
topic 0
0.017*"" + 0.014*"，" + 0.013*"到" + 0.013*"的" + 0.012*"。" + 0.011*"了" + 0.009*"我" + 0.009*"这" + 0.008*"晚" + 0.008*"你"
topic 1
0.017*"。" + 0.015*"的" + 0.014*"有" + 0.012*"这" + 0.011*"，" + 0.011*"了" + 0.010*"是" + 0.009*"到" + 0.009*"年" + 0.009*"我"
topic 2
0.012*"在" + 0.011*"就" + 0.011*"的" + 0.011*"" + 0.011*"了" + 0.010*"。" + 0.010*"是" + 0.010*"到" + 0.009*"又" + 0.009*"过"
